# Load data from backend

In [ ]:
import requests
import json

headers = {'Cookie': '_orca_session=C1iXOgy3Ot1RBjOoS8Jlk%2Fnco4XfPEl00dvzT898KjlrQCVGyVkTg1QX5lh1Bs1Up4mTHsyGkzJ43s18envJxLmoadBxo0Cn58Y6m4YP0ru3dR9jv5mhsKHfeYxCwYwAiaFKJ%2FgkU5MW%2Fa0n8gMXJHHnn1aNvLZFPDMEMagTbAaFnXpManKfUTr%2F8sfkFkxD1S2Ev%2FeBd4ZRhcL5CumwbAqxkiRk%2F53EkSczz8d8%2FWOMAQJ%2Bz01zEsBQ1PVp1Q2h97NMbmAW1cwH2hH7xO7DIGzeS6S57HFLgH1FHC4P2W%2FNEuYoboHzqQJEkHh1qA8wtd%2BFO3%2BGpENxyDxPnl7dmtnpeHa9--Dm3Fwrgb8nKYxNBq--jJAWAAqO5%2FmcCmyDae7HzQ%3D%3D'}
UNIT_URL = "https://orca-develop.herokuapp.com/units.json"
ACTIVTY_URL = "https://orca-develop.herokuapp.com/activities.json"

# get activities from orca
r = requests.get(ACTIVTY_URL, headers=headers)
activities = r.json()

r = requests.get(UNIT_URL, headers=headers)
units = r.json()

# Class Definitions

In [ ]:
class Activity:
  def __init__(self, id, label, category, activity_executions):
    self.id = id
    self.label = label
    self.category = category
    self.activity_executions = activity_executions

  def __repr__(self):
    return "id: " + str(self.id) + "\n label: " + self.label

  # executions which are yet not full
  def getAvailableExecutions(self, unit):
    def singleMovaActivityPerDay(unit, activity_execution):
      for unit_activity_execution in unit.assigned_activities_executions:
        if unit_activity_execution.starts_at <= activity_execution.starts_at and \
            unit_activity_execution.ends_at >= activity_execution.starts_at or \
            unit_activity_execution.starts_at <= activity_execution.ends_at and \
            unit_activity_execution.ends_at >= activity_execution.ends_at:
            return False
      return True

    def noOutOfCampAcitivities(unit, activity_execution):
      if unit.starts_at <= activity_execution.starts_at and unit.ends_at >= activity_execution.ends_at:
        return True
      else:
        return False
         
    possible_activity_executions = []
    for activity_execution in self.activity_executions:
      print(activity_execution.hasLanguage(unit.language))
      print(activity_execution.amount_participants > unit.participant_count)
      print(singleMovaActivityPerDay(unit, activity_execution))
      print(noOutOfCampAcitivities(unit, activity_execution))
      print(possible_activity_executions.append(activity_execution))
                                            
      if activity_execution.hasLanguage(unit.language) and  \
          activity_execution.amount_participants > unit.participant_count and  \
          singleMovaActivityPerDay(unit, activity_execution) and  \
          noOutOfCampAcitivities(unit, activity_execution) and \
          transportCapcitynotOverreached() and \    # respect transport only with ausflug
          maxMovaBookingsCheck()   # is activity still allowed to be assigned with respect to mova rules

        possible_activity_executions.append(activity_execution)
    
    return possible_activity_executions

In [ ]:
class ActivityExecutions:
  def __init__(self, id, amount_participants, starts_at, ends_at, languages, spot, transport):
    self.id = id
    self.amount_participants = amount_participants
    self.starts_at = starts_at
    self.ends_at = ends_at
    self.languages = languages
    self.spot = spot
    self.transport = transport

    self.assigned_units = []

  def addUnit(self, unit):
    self.assigned_units.append(unit)

  def hasLanguage(self, language):
    if language in self.languages:
      return True
    else:
      return False

In [ ]:
class Unit:
  def __init__(self, id, district, starts_at, ends_at, language, participant_count, leader_count, stufe, selected_activities):
    self.id = id
    self.district = district
    self.starts_at = starts_at
    self.ends_at = ends_at
    self.language = language
    self.participant_count = participant_count
    self.leader_count = leader_count
    self.stufe = stufe
    self.selected_activities = selected_activities
    self.assigned_activities_executions = []

  def __repr__(self):
    return "id" + str(self.id)

  def getHappyScore(self):
    happyScore = 0
    
    return happyScore

  def satisfied():
      # mova rules
      #

  def getNextBooking(self):
    # satisfied and next highest priority booking
    for selected_activity in self.selected_activities:
      if not any(x.id == selected_activity.id for x in self.assigned_activities_executions) and satisfied():
        return selected_activity
    
    return []

  def addActivityExecutions(self, activity_execution):
    self.assigned_activities_executions.append(activity_execution)

In [ ]:
class SelectedActivity:
  def __init__(self, id, priority):
    self.id = id
    self.priority = priority

In [ ]:
class Scheduler:
  def __init__(self, ):
    self.acitivites = []
    self.units = []
  
  def addActivity(self, activity):
    self.acitivites.append(activity)

  def addUnit(self, unit):
    self.units.append(unit)
  
  def getHappyScore(self):
    return sum([unit.getHappyScore() for unit in self.units]) / len(self.units)

  def getActivity(self, activity_id):
    for activity in self.acitivites:
      if activity.id == activity_id:
        return activity

  def schedule(self, iterations):
    # iterate, always start with "worst unit so far"
    for n in range(iterations):
      units_sorted = sorted(self.units, key=Unit.getHappyScore, reverse=True)

      for unit_sorted in units_sorted:
        # get activity which unit has selected and has not yet been assigned, get one with highest prio
        selected_activity = unit_sorted.getNextBooking()

        if selected_activity:
          # get executions of that activity and check which execution the unit can attend based:
          #                                                                                     - unit size
          #                                                                                     - unit language
          #                                                                                     - unit availability (camp date)
          #                                                                                     - unit no overlapping other acitivities
          activity = self.getActivity(selected_activity.id)

          if activity: 
            available_executions = activity.getAvailableExecutions(unit_sorted)
          
            if available_executions: 
              # assign the unit to the activity
              unit.assignAcitivtyExecutions(available_executions)
              available_executions.addUnit(unit_sorted)
              print("Assigned")
            else:
              print("Assignment for activity " + str(selected_activity.id) + " and unit " + str(unit_sorted.id) + " not possible")
          else:
            print("Activity " + str(selected_activity.id) + " not found")
        else:
          print("Unit has all selected activities assigned or no activity executions")

      # units which are unsatisfied, because booking possibilities didn't or didn't make any bookings

# Load scheduler with data

In [ ]:
scheduler = Scheduler()

# create activities
for activity in activities:
  act_executions = activity.get("activity_executions")

  executions = []
  for act_exe in act_executions:
    # respect already assigend units to executions
    executions.append(ActivityExecutions(
                                         id = act_exe.get("id"), 
                                         amount_participants = act_exe.get("amount_participants"), 
                                         starts_at = act_exe.get("starts_at"), 
                                         ends_at = act_exe.get("ends_at"), 
                                         languages = act_exe.get("languages"), 
                                         spot = act_exe.get("spot"), 
                                         transport = act_exe.get("transport")))

  act = Activity(activity.get("id"), activity.get("activity_category").get("code"), executions)
  scheduler.addActivity(act)

# create units
for unit in units:
  sel_activities = unit.get("unit_activities")

  selected_activities = []
  for sel_act in sel_activities:
    selected_activities.append(SelectedActivity(sel_act.get("activity_id"), sel_act.get("priority")))

  selected_activities = sorted(selected_activities, key=lambda selected_activity: selected_activity.priority)

  un = Unit(
          id = unit.get("id"),
          district = unit.get("district"),
          starts_at = unit.get("starts_at"),
          ends_at = unit.get("ends_at"),
          language = unit.get("language"),
          participant_count = unit.get("participant_count"),
          stufe = unit.get("stufe"),
          selected_activities = selected_activities
  )

  scheduler.addUnit(un)

# Scheduling

In [ ]:
scheduler.schedule(1)

203
[]
Assignment for activity 6 and unit 203 not possible
Unit has all selected activities assigned or no activity executions
205
[]
Assignment for activity 117 and unit 205 not possible
206
[]
Assignment for activity 6 and unit 206 not possible
Unit has all selected activities assigned or no activity executions
Unit has all selected activities assigned or no activity executions


# Get Happy Score

In [ ]:
# get current assignment happy score
scheduler.getHappyScore()

0.0